In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [ ]:
path_to_file = tf.keras.utils.get_file("shakespeare.txt",
                                       "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

text = open(path_to_file, "r", encoding="utf-8").read()
lines = text.split("\n")


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
total_words = len(tokenizer.word_index) + 1
print(total_words)

input_sequences = []

for line in lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1]) 

max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding="pre")

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

embedding_dim = 50
embedding_index = {}

with open("glove.6B.50d.txt", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        embedding_index[word] = vector

embedding_matrix = np.zeros((total_words, embedding_dim))

for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
model = Sequential([
    Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length-1, trainable=True),
    LSTM(256, return_sequences=True),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(total_words, activation="softmax")
]) ##LSTM

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:

model_cnn_lstm = Sequential([
    Embedding(total_words, embedding_dim, input_length=max_sequence_length-1, trainable=True),
    Conv1D(filters=256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(256, return_sequences=True),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(total_words, activation="softmax")
]) ##CNN+LSTM

model_cnn_lstm.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
model.fit(X, y, epochs=100, verbose=1, batch_size=64)
# model_cnn_lstm.fit(X, y, epochs=10, verbose=1, batch_size=64)


In [ ]:
def generate_text(seed_text, next_words=20):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding="pre")
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        seed_text += " " + output_word
    return seed_text


In [ ]:
print(generate_text("The night was dark", next_words=20))
